In [11]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd

# OLD INVESTIGATION

In [4]:
path = '../../data/CDCdata/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv'
print(os.stat(path).st_size / 1000 / 1000, 'MB')

3928.479796 MB


In [12]:
chunksize=250000
raw_iter = pd.read_csv(path, chunksize=chunksize)

In [13]:
chunk = next(raw_iter)

In [14]:
chunk[100:110]

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
100,2020-12,ID,16,NEZ PERCE,16069.0,65+ years,Female,Missing,Missing,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,NaN,NaN
101,2020-11,TN,47,SCOTT,47151.0,65+ years,Female,NaN,NaN,0.0,NaN,Missing,Yes,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN
102,2021-01,MT,30,LEWIS AND CLARK,30049.0,0 - 17 years,Male,Missing,Missing,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,NaN,NaN
103,2020-12,IL,17,PERRY,17145.0,0 - 17 years,Male,NaN,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,Missing,NaN
104,2020-12,NY,36,SARATOGA,36091.0,0 - 17 years,Male,Missing,NaN,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
105,2020-09,IL,17,ADAMS,17001.0,18 to 49 years,Male,Missing,Missing,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,Missing,NaN
106,2021-04,MN,27,BECKER,27005.0,18 to 49 years,Male,Unknown,Missing,0.0,0.0,Missing,Yes,Laboratory-confirmed case,Symptomatic,Missing,Missing,No,NaN
107,2021-03,CO,8,LA PLATA,8067.0,18 to 49 years,Male,Missing,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
108,2020-07,KS,20,MCPHERSON,20113.0,18 to 49 years,Male,NaN,NaN,1.0,0.0,Clinical evaluation,Missing,Laboratory-confirmed case,Symptomatic,No,No,No,Yes
109,2020-10,MN,27,MORRISON,27097.0,18 to 49 years,Male,Unknown,Missing,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN


In [15]:
chunk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 19 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   case_month                       250000 non-null  object 
 1   res_state                        250000 non-null  object 
 2   state_fips_code                  250000 non-null  int64  
 3   res_county                       249182 non-null  object 
 4   county_fips_code                 249182 non-null  float64
 5   age_group                        237646 non-null  object 
 6   sex                              208438 non-null  object 
 7   race                             136223 non-null  object 
 8   ethnicity                        117317 non-null  object 
 9   case_positive_specimen_interval  81345 non-null   float64
 10  case_onset_interval              123475 non-null  float64
 11  process                          250000 non-null  object 
 12  ex

In [16]:
chunk.replace('Missing', np.nan, inplace=True)

In [17]:
chunk['case_month'] = pd.to_datetime(chunk['case_month'], format='%Y-%m')

In [18]:
chunk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 19 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   case_month                       250000 non-null  datetime64[ns]
 1   res_state                        250000 non-null  object        
 2   state_fips_code                  250000 non-null  int64         
 3   res_county                       249182 non-null  object        
 4   county_fips_code                 249182 non-null  float64       
 5   age_group                        235220 non-null  object        
 6   sex                              207803 non-null  object        
 7   race                             113556 non-null  object        
 8   ethnicity                        85517 non-null   object        
 9   case_positive_specimen_interval  81345 non-null   float64       
 10  case_onset_interval              123475 non-

Looking into a bit, there doesn't seem to be any rhyme or reason to the cases with missing counties, like they are spread out somewhat.

**Drop rows with both missing 'res_county' and missing 'county_fips_code'** 

In [20]:
# For the 800+ cases with missing counties, we presumedly cannot use these as we need to put them in a county.
# How spread out are they (are they concentrated in a particular state, e.g.)?
chunk[chunk['county_fips_code'].isna()]['res_state'].value_counts()

AZ    164
IN    164
PA    163
MD    163
GA     82
IL     82
Name: res_state, dtype: int64

In [25]:
chunk[chunk['county_fips_code'].isna()].groupby(['case_month'])['res_state'].value_counts().sort_index()

case_month  res_state
2020-03-01  IN           11
2020-05-01  AZ           25
            MD           21
2020-06-01  AZ           12
2020-07-01  AZ           12
            GA           66
            IN           28
2020-09-01  IL           13
            PA           11
2020-10-01  IN           41
            MD           18
            PA           49
2020-11-01  AZ           36
            GA           16
            IL           26
            IN           11
            MD           10
2020-12-01  AZ           13
            IL           43
            IN           54
            PA           14
2021-01-01  AZ           12
            IN           19
            MD           29
            PA           38
2021-02-01  AZ           23
            MD           18
            PA           12
2021-03-01  AZ           16
            MD           18
2021-04-01  MD           49
            PA           21
2021-05-01  PA           18
2021-07-01  AZ           15
Name: res_state, dtype: in

In [10]:
pd.DataFrame([(col, len(chunk[col].unique())) for col in chunk.columns], columns=['feature', 'unique values'])

,feature,unique values
0,case_month,19
1,res_state,49
2,state_fips_code,49
3,res_county,1034
4,county_fips_code,1531
5,age_group,5
6,sex,4
7,race,8
8,ethnicity,4
9,case_positive_specimen_interval,78


Hmmm, we know the US has over 3000 counties, but only 1531 county codes are in this (early!) chunk of 250,000 covid cases. There are 29M+ records total, so it will be interesting to see what additional counties eventually report.

In [41]:
chunk['age_group'].value_counts()

18 to 49 years    127994
50 to 64 years     41856
0 - 17 years       41094
65+ years          24276
Name: age_group, dtype: int64

In [30]:
chunk['case_month'].value_counts().sort_index()

2020-01-01       17
2020-02-01       19
2020-03-01     2736
2020-04-01     6988
2020-05-01     8407
2020-06-01     9947
2020-07-01    15679
2020-08-01    13477
2020-09-01    12375
2020-10-01    19145
2020-11-01    35774
2020-12-01    37494
2021-01-01    30997
2021-02-01    15522
2021-03-01    12863
2021-04-01    13418
2021-05-01     8289
2021-06-01     3585
2021-07-01     3268
Name: case_month, dtype: int64

In [23]:
print('Nan values:', len(chunk[chunk['current_status'].isna()]))
print(chunk['current_status'].value_counts())

Nan values: 0
Laboratory-confirmed case    217847
Probable Case                 32153
Name: current_status, dtype: int64


In [24]:
print('Nan values:', len(chunk[chunk['hosp_yn'].isna()]))
print(chunk['hosp_yn'].value_counts())

Nan values: 111258
No         103836
Unknown     26455
Yes          8451
Name: hosp_yn, dtype: int64


In [25]:
print('Nan values:', len(chunk[chunk['icu_yn'].isna()]))
print(chunk['icu_yn'].value_counts())

Nan values: 207063
Unknown    27017
No         15148
Yes          772
Name: icu_yn, dtype: int64


In [26]:
print('Nan values:', len(chunk[chunk['death_yn'].isna()]))
print(chunk['death_yn'].value_counts())

Nan values: 122557
No         106875
Unknown     20443
Yes           125
Name: death_yn, dtype: int64


In [30]:
len(chunk[(chunk['current_status'] == 'Probable Case') & chunk['hosp_yn'].isna() & chunk['icu_yn'].isna() & chunk['death_yn'].isna()])

11936

In [31]:
len(chunk[(chunk['current_status'] == 'Probable Case') | chunk['hosp_yn'].isna() | chunk['icu_yn'].isna() | chunk['death_yn'].isna()])

219062

In [33]:
print('missing hosp, icu, and death data:', len(chunk[chunk['hosp_yn'].isna() & chunk['icu_yn'].isna() & chunk['death_yn'].isna()]))
print('missing hosp, icu, or death data:', len(chunk[chunk['hosp_yn'].isna() | chunk['icu_yn'].isna() | chunk['death_yn'].isna()]))

missing hosp, icu, and death data: 78177
missing hosp, icu, or death data: 214412


In [40]:
chunk[chunk['icu_yn'].isin(['Yes']) & chunk['death_yn'].isin(['Yes'])]

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
109526,2020-03-01,LA,22,JEFFERSON,22051.0,65+ years,Male,White,Non-Hispanic/Latino,0.0,NaN,NaN,Yes,Laboratory-confirmed case,Symptomatic,Yes,Yes,Yes,Yes
124618,2020-04-01,CA,6,ALAMEDA,6001.0,65+ years,Female,White,Non-Hispanic/Latino,NaN,0.0,NaN,NaN,Laboratory-confirmed case,Symptomatic,Yes,Yes,Yes,NaN
127671,2020-04-01,CA,6,ALAMEDA,6001.0,65+ years,Female,White,Non-Hispanic/Latino,NaN,NaN,NaN,NaN,Laboratory-confirmed case,Symptomatic,Yes,Yes,Yes,NaN
174636,2021-01-01,OH,39,LOGAN,39091.0,65+ years,Male,White,Non-Hispanic/Latino,NaN,0.0,Clinical evaluation,Yes,Probable Case,Symptomatic,Yes,Yes,Yes,Yes
175674,2021-01-01,PA,42,NaN,NaN,65+ years,Male,White,NaN,0.0,0.0,NaN,Yes,Laboratory-confirmed case,Symptomatic,Yes,Yes,Yes,Yes
188389,2021-01-01,OH,39,LOGAN,39091.0,65+ years,Male,White,Non-Hispanic/Latino,NaN,0.0,Clinical evaluation,Yes,Probable Case,Symptomatic,Yes,Yes,Yes,Yes
202773,2020-04-01,PA,42,COLUMBIA,42037.0,65+ years,Female,NaN,NaN,0.0,0.0,NaN,Yes,Laboratory-confirmed case,Symptomatic,Yes,Yes,Yes,Yes


## Looks like there are 52 reporting periods and each hospital was supposed to submit one report per week (which 4669 of 5051 did). 

In [21]:
print(hospital_raw['collection_week'].nunique())
hospital_raw['collection_week'].value_counts().sort_index()

52


2020/07/31    4886
2020/08/07    4895
2020/08/14    4849
2020/08/21    4860
2020/08/28    4902
2020/09/04    4922
2020/09/11    4923
2020/09/18    4925
2020/09/25    4946
2020/10/02    4964
2020/10/09    4974
2020/10/16    4982
2020/10/23    4991
2020/10/30    4997
2020/11/06    5003
2020/11/13    5007
2020/11/20    5009
2020/11/27    5011
2020/12/04    5016
2020/12/11    5020
2020/12/18    5018
2020/12/25    5021
2021/01/01    5020
2021/01/08    5013
2021/01/15    5017
2021/01/22    5018
2021/01/29    5017
2021/02/05    5017
2021/02/12    5018
2021/02/19    5017
2021/02/26    5019
2021/03/05    5018
2021/03/12    5017
2021/03/19    5017
2021/03/26    5016
2021/04/02    5015
2021/04/09    5016
2021/04/16    5014
2021/04/23    5016
2021/04/30    5016
2021/05/07    5004
2021/05/14    5009
2021/05/21    5009
2021/05/28    5010
2021/06/04    5009
2021/06/11    5008
2021/06/18    5006
2021/06/25    5003
2021/07/02    5002
2021/07/09    5006
2021/07/16    5005
2021/07/23    5006
Name: collec

In [310]:
# list of report counts filed by individual hospitals
hospital_raw.groupby(['hospital_name', 'state'])['hospital_pk'].count().value_counts().sort_index()

1         9
2         3
3         3
4         2
5         1
7         2
8         6
9         3
10        1
11        2
12        1
13        2
14        4
15        2
16        1
22        2
23        2
26        3
28        5
30        1
31        1
32        2
33        4
34        2
35        6
37        5
39        5
40       14
41       12
42        9
43       10
44       21
45       17
46       11
47       21
48       30
49       15
50       45
51       85
52     4669
90        1
102       1
104       7
156       3
Name: hospital_pk, dtype: int64

In [270]:
hospital_raw.groupby('hospital_pk').nunique()['fips_code'].value_counts()

1    5051
0      16
Name: fips_code, dtype: int64

In [271]:
hospital_raw.groupby('hospital_name').nunique()['fips_code'].value_counts()

1     4814
2       56
0       16
3       14
4        9
5        6
13       1
Name: fips_code, dtype: int64

In [164]:
type(hospital_raw['geocoded_hospital_address'][0])

str

In [227]:
hospital['geocoded_hospital_address'][15]

'POINT (-85.823353 33.657123)'

In [228]:
hospital.isna().sum()

hospital_pk                                                       0
collection_week                                                   0
state                                                             0
zip                                                               0
geocoded_hospital_address                                     21158
previous_day_admission_adult_covid_confirmed_7_day_sum         1442
previous_day_admission_pediatric_covid_confirmed_7_day_sum     6333
previous_day_admission_adult_covid_suspected_7_day_sum         1516
previous_day_admission_pediatric_covid_suspected_7_day_sum     6133
dtype: int64